# Imports

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
# figure settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set_theme(style="whitegrid")
sns.set_palette(palette="Paired")

# set seed
np.random.seed(42)

# Load Data

In [3]:
X_train = pd.read_csv("../../data/processed/train_data_cleaned.csv")
y_train = pd.read_csv("../../data/processed/train_labels.csv")

In [4]:
X_train = X_train.iloc[:100000] # max 260601
y_train = y_train.iloc[:100000] # max 260601

# Random Forest

In [5]:
# Build random forest classifier
rf = RandomForestClassifier(n_estimators=100,
                            max_depth=20,
                            max_features="sqrt",
                            random_state=42,
                            n_jobs=-1,
                            verbose=1)

In [6]:
# Perform cross validation
cv_results = cross_validate(rf,
                            X_train,
                            y_train["damage_grade"],
                            cv=5,
                            scoring=("accuracy", "matthews_corrcoef"),
                            return_train_score=True,
                            n_jobs=-1,
                            verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   12.3s finished


In [7]:
# Display results
cv_results = pd.DataFrame(cv_results).reset_index(names="k-fold")
display(cv_results)

# Calculate mean and standard deviation
cv_acc_mean_rf = cv_results["test_accuracy"].mean()
cv_acc_std_rf = cv_results["test_accuracy"].std()
print(f"Mean Accuracy: {cv_acc_mean_rf:.3f} +/- {cv_acc_std_rf:.3f}")

cv_mcc_mean_rf = cv_results["test_matthews_corrcoef"].mean()
cv_mcc_std_rf = cv_results["test_matthews_corrcoef"].std()
print(f"Mean MCC: {cv_mcc_mean_rf:.3f} +/- {cv_mcc_std_rf:.3f}")

,k-fold,fit_time,score_time,test_accuracy,train_accuracy,test_matthews_corrcoef,train_matthews_corrcoef
0,0,4.008131,3.858672,0.70305,0.896463,0.438359,0.813120
1,1,7.850360,0.147517,0.69855,0.898988,0.429471,0.817731
2,2,3.998116,3.866681,0.70215,0.894900,0.435110,0.809776
3,3,7.823085,0.171135,0.70195,0.897600,0.435248,0.815072
4,4,8.128152,0.491601,0.69795,0.897388,0.427764,0.814421


Mean Accuracy: 0.701 +/- 0.002
Mean MCC: 0.433 +/- 0.004


# Xgboost

In [9]:
# Build Xgboost classifier
xgb = XGBClassifier(n_estimators=100,
                    max_depth=20,
                    learning_rate=0.1,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    random_state=42,
                    n_jobs=-1,
                    verbosity=1)

In [10]:
# Label encode y_train for Xgboost (expects 0, 1, 2, not 1, 2, 3)
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train["damage_grade"].to_numpy())

In [28]:
# Perform cross validation
cv_results_xgb = cross_validate(xgb,
                                X_train,
                                y_train_encoded,
                                cv=5,
                                scoring=("accuracy", "matthews_corrcoef"),
                                return_train_score=True,
                                n_jobs=-1,
                                verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.0min finished


In [29]:
# Display results
cv_results_xgb = pd.DataFrame(cv_results_xgb).reset_index(names="k-fold")
display(cv_results_xgb)

# Calculate mean and standard deviation
cv_acc_mean_xgb = cv_results_xgb["test_accuracy"].mean()
cv_acc_std_xgb = cv_results_xgb["test_accuracy"].std()
print(f"Mean Accuracy: {cv_acc_mean_xgb:.3f} +/- {cv_acc_std_xgb:.3f}")

cv_mcc_mean_xgb = cv_results_xgb["test_matthews_corrcoef"].mean()
cv_mcc_std_xgb = cv_results_xgb["test_matthews_corrcoef"].std()
print(f"Mean MCC: {cv_mcc_mean_xgb:.3f} +/- {cv_mcc_std_xgb:.3f}")

,k-fold,fit_time,score_time,test_accuracy,train_accuracy,test_matthews_corrcoef,train_matthews_corrcoef
0,0,105.755020,2.491858,0.72300,0.997487,0.481057,0.995475
1,1,106.388875,2.590602,0.72515,0.997200,0.485294,0.994957
2,2,105.457865,2.585622,0.72260,0.997112,0.478781,0.994799
3,3,106.441460,2.501519,0.71885,0.997050,0.472464,0.994687
4,4,105.634036,2.472586,0.72065,0.996862,0.476448,0.994349


Mean Accuracy: 0.722 +/- 0.002
Mean MCC: 0.479 +/- 0.005


# Comparison Plot

In [30]:
# Melt the dataset to create a long-form version for plotting
df_melt_rf = cv_results[["k-fold", "test_accuracy", "test_matthews_corrcoef"]].melt(id_vars='k-fold', var_name='Metric', value_name='Score')
df_melt_xgb = cv_results_xgb[["k-fold", "test_accuracy", "test_matthews_corrcoef"]].melt(id_vars='k-fold', var_name='Metric', value_name='Score')

In [35]:
# Create figure and axes
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

# Plot a stacked bar charts using Seaborn
# Random Forest
palette = {
    "test_accuracy": sns.color_palette("Paired")[0],
    "test_matthews_corrcoef": sns.color_palette("Paired")[2]
}
fig_ax1 = sns.barplot(x='k-fold', y='Score', hue='Metric', data=df_melt_rf, ax=axes[0], dodge=False, errorbar=None, palette=palette)
axes[0].set_title("Test Scoring RF" + "\n" +
                  f"Mean MCC: {cv_mcc_mean_rf:.3f} +/- {cv_mcc_std_rf:.3f}, Mean Accuracy: {cv_acc_mean_rf:.3f} +/- {cv_acc_std_rf:.3f}")
axes[0].legend().set_visible(False)
# XGBoost
fig_ax2 = sns.barplot(x='k-fold', y='Score', hue='Metric', data=df_melt_xgb, ax=axes[1], dodge=False, errorbar=None, palette=palette)
axes[1].set_title("Test Scoring XGBoost" + "\n" +
                  f"Mean MCC: {cv_mcc_mean_xgb:.3f} +/- {cv_mcc_std_xgb:.3f}, Mean Accuracy: {cv_acc_mean_xgb:.3f} +/- {cv_acc_std_xgb:.3f}")
axes[1].legend().set_visible(False)

# Add horizontal guidelines for interpretation
# Get x min and x max
xmin, xmax = axes[0].get_xlim()
# Performance guide lines
axes[0].hlines(y=0.5, xmin=xmin, xmax=xmax, linestyles="dotted", colors='r', label="Random Guessing")
axes[1].hlines(y=0.5, xmin=xmin, xmax=xmax, linestyles="dotted", colors='r', label="Random Guessing")
# Mean guidelines
axes[0].hlines(y=cv_mcc_mean_rf, xmin=xmin, xmax=xmax, linestyles="dashed", colors=sns.color_palette("Paired")[3], label="Mean MCC")
axes[1].hlines(y=cv_mcc_mean_xgb, xmin=xmin, xmax=xmax, linestyles="dashed", colors=sns.color_palette("Paired")[3], label="Mean MCC")
axes[0].hlines(y=cv_acc_mean_rf, xmin=xmin, xmax=xmax, linestyles="dashed", colors=sns.color_palette("Paired")[1], label="Mean Accuracy")
axes[1].hlines(y=cv_acc_mean_xgb, xmin=xmin, xmax=xmax, linestyles="dashed", colors=sns.color_palette("Paired")[1], label="Mean Accuracy")

plt.suptitle("Cross Validation Results (100000 samples)", fontweight='bold')
plt.setp(axes, ylim=(0.3, 0.8))
handles, labels = axes[0].get_legend_handles_labels()
order = [0, 2, 3, 1, 4]
plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order], bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)

plt.tight_layout()
plt.show()